In [24]:
import pandas as pd
import numpy as np
import requests
from config import  gkey
import gmaps
import time
import json

In [25]:
#import csv files into notebook
#combined Census, zillow and Open weather data
combined_data = pd.read_csv("Combined_loc_CA.csv")
combined_data.head()

,Zipcode,City,State,CountyName,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate,average rent,Median Home Value,Number of House Build in year,Avg_HomeValue,Employment Rate,Latitude,Longitude,Temp
0,90001,Florence-Graham,CA,Los Angeles,57942.0,28.0,34323.0,11294.0,18902.0,32.622277,915.0,272200.0,1950.0,2.999500e+05,44.482413,33.97,-118.25,82.02
1,90002,Los Angeles,CA,Los Angeles,51826.0,27.5,32520.0,11212.0,18302.0,35.314321,902.0,248200.0,1951.0,2.946667e+05,40.938911,34.05,-118.24,82.62
2,90003,Los Angeles,CA,Los Angeles,70208.0,28.0,31878.0,10611.0,24556.0,34.976071,958.0,268500.0,1951.0,3.002417e+05,42.456700,34.05,-118.24,82.62
3,90004,Los Angeles,CA,Los Angeles,63095.0,35.5,43180.0,29194.0,12919.0,20.475473,1044.0,838000.0,1946.0,1.168492e+06,57.777954,34.05,-118.24,82.62
4,90005,Los Angeles,CA,Los Angeles,39338.0,33.9,31485.0,20265.0,11520.0,29.284661,943.0,672600.0,1947.0,7.051917e+05,58.897249,34.05,-118.24,82.62


In [26]:
#california whole foods market location data
whole_foods_df = pd.read_csv("Whole Foods CA Location.csv")
whole_foods_df.head()

,Name,Latitude,Longitude,Address
0,Whole Foods Market,34.045870,-118.257590,"788 S Grand Ave, Los Angeles"
1,Whole Foods Market,33.976712,-118.418102,"12746 W Jefferson Blvd, Playa Vista"
2,Whole Foods Market,34.022625,-118.438046,"11666 National Blvd, Los Angeles"
3,Whole Foods Market,34.061229,-118.446954,"1050 Gayley Ave, Los Angeles"
4,Whole Foods Market,34.053639,-118.467407,"11737 San Vicente Blvd, Los Angeles"


In [27]:
#california walmart location data
walmart_df = pd.read_csv("Walmart CA Location.csv")
walmart_df.head()

,Name,Latitude,Longitude,Address
0,Walmart Supercenter,33.954813,-118.187613,"4651 Firestone Blvd, South Gate"
1,Walmart Supercenter,33.910008,-118.208162,"2100 N Long Beach Blvd, Compton"
2,Walmart,33.956942,-118.272099,"150 E 88th St, Los Angeles"
3,Walmart Supercenter,34.050425,-118.085373,"1827 Walnut Grove Ave, Rosemead"
4,Walmart Supercenter,33.984986,-118.104991,"8500 Washington Blvd, Pico Rivera"


# Heatmap - Median Home Value by Location

In [28]:
# Configure gmaps with API key
gmaps.configure(api_key=gkey)

In [29]:
# Store 'Lat' and 'Lng' into  locations 
locations = combined_data[["Latitude", "Longitude"]].astype(float)

# Convert Home Value/ Population to float and store
# HINT: be sure to handle NaN values
home_value = combined_data["Median Home Value"].astype(float)
population = combined_data["Population"].astype(float)
#max home value and poplulation
print(population.max())
print(home_value.max())

106360.0
2000001.0


In [31]:
# Create a home value Heatmap layer
fig = gmaps.figure()
# Customize the size of the figure
figure_layout = {
    'width': '1000px',
    'height': '700px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

heat_layer = gmaps.heatmap_layer(locations, weights=home_value, 
                                 dissipating=False, max_intensity=2000001,
                                 point_radius = 0.08)

# Adjust heat_layer setting to help with heatmap dissipating on zoom
heat_layer.dissipating = False
heat_layer.max_intensity = 2000001
heat_layer.point_radius = 0.05

fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(border='1px solid black', height='700px', margin='0 auto 0 auto', padding='1px', wi…

# Heatmap - Population by Location

In [32]:
# Create a population Heatmap layer
fig_1 = gmaps.figure()
# Customize the size of the figure
figure_layout = {
    'width': '700px',
    'height': '1000px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig_1 = gmaps.figure(layout=figure_layout)


heat_layer_1 = gmaps.heatmap_layer(locations, weights=population, 
                                 dissipating=False, max_intensity=106360,
                                 point_radius = 0.08)

# Adjust heat_layer setting to help with heatmap dissipating on zoom
heat_layer_1.dissipating = False
heat_layer_1.max_intensity = 106360
heat_layer_1.point_radius = 0.05

fig_1.add_layer(heat_layer_1)

fig_1

Figure(layout=FigureLayout(border='1px solid black', height='1000px', margin='0 auto 0 auto', padding='1px', w…

# Mark Whole Foods Market Location on Home Value Heatmap

In [15]:
# Create a list containing coordinates
latitude_wf = whole_foods_df["Latitude"]
latitude_wf = latitude_wf.values
longitude_wf = whole_foods_df["Longitude"]
longitude_wf= longitude_wf.values
coordinates_wf = list(zip(latitude_wf,longitude_wf))
print(len(coordinates_wf))
#mark whole foods locations on fig
wf_markers = gmaps.marker_layer(coordinates_wf)
fig.add_layer(wf_markers)
fig

79


Figure(layout=FigureLayout(border='1px solid black', height='1000px', margin='0 auto 0 auto', padding='1px', w…

# Mark Walmart Location on Home Value Heatmap

In [16]:
# Create a list containing coordinates
latitude_wl = walmart_df["Latitude"]
latitude_wl = latitude_wl.values
longitude_wl = walmart_df["Longitude"]
longitude_wl = longitude_wl.values
coordinates_wl = list(zip(latitude_wl,longitude_wl))
print(len(coordinates_wl))


281


In [21]:
# Create walmart symbol layer
walmart_layer = gmaps.symbol_layer(
    coordinates_wl, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=5
)
#mark walmart locations on fig
fig.add_layer(walmart_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='1000px', margin='0 auto 0 auto', padding='1px', w…

# Pin Walmart Location on Population Heatmap

In [19]:
#mark walmart locations on population fig
fig_1.add_layer(walmart_layer)
fig_1

Figure(layout=FigureLayout(border='1px solid black', height='1000px', margin='0 auto 0 auto', padding='1px', w…